In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
# Load T0 model and tokenizer
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/Train/Train_Anticipation_All.csv")
test_dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2/MViTv2_Test_Anticipation.csv")
print(dataset)


In [ ]:
def preprocess_data(examples):
    inputs = examples["context"]
    print(inputs)
    targets = examples["answer"]
    model_inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=1024)
    labels = tokenizer(targets, padding="max_length", truncation=True, max_length=25)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

dataset = dataset.map(preprocess_data, batched=True)
test_dataset = test_dataset.map(preprocess_data, batched = True)
print(dataset)


In [ ]:
num_epochs = 1 
from transformers import TrainingArguments, Trainer
for i in range(num_epochs):

    training_args = TrainingArguments(
        output_dir="./ul2_qa",
        evaluation_strategy="epoch",
        save_strategy="no",
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=30,
        weight_decay=0.01,
        logging_dir="./logs",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset["train"],
        eval_dataset = test_dataset["train"],
        # predict_with_generate = True,
        # preprocess_logits_for_metrics=preprocess_logits_for_metrics
        # compute_loss_func=compute_loss_func
    )

    trainer.train()


In [ ]:
model.save_pretrained("./t5_small_anticipation")
tokenizer.save_pretrained("./t5_small_anticipation")

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
import torch

model_name = "./t5_small_anticipation"

# model_name = "t5_small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to("cuda")
def generate_answer(context=""):
    input_text = context
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")
    output_ids = model.generate(input_ids)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

In [ ]:
test_dataset = load_dataset("csv", data_files="/Users/eddie/Downloads/VLMTrain/MViTv2/MViTv2_Test_Anticipation.csv")
test_dataset = test_dataset["train"]
# questions = test_dataset["question"]
context = test_dataset["context"]
labels = test_dataset["answer"]    
print(test_dataset)
print(labels)
print(context)

In [ ]:
generated_answers = []
for i in range(len(context)):
    # print(questions[i], context[i])
    generated_answers.append(generate_answer(context[i]))
    print(generated_answers[i])
print(generated_answers)
print(labels)

In [ ]:
correct = 0
total = 0
for i in range(len(generated_answers)):
    if((generated_answers[i])==(labels[i])):
        correct = correct+1
        print(correct, total)
    total = total+1
print(correct/total)